In [ ]:
%pip install langchain pyautogen

In [14]:
import autogen
import os
from typing_extensions import Annotated

GROQ_API_KEY = "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"

config_llamacpp = {'model': 'llama-3-8b', 'base_url': 'http://llm:8000/v1/', 'api_key': "NULL"}

llm_config = config_llamacpp
llm_config['cache_seed'] = None

logging_session_id = autogen.runtime_logging.start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config
)

is_terminate_any = lambda x: x["content"] and "TERMINATE" in x["content"]
is_terminate_end = lambda x: x["content"] and x["content"].strip().endswith("TERMINATE")
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=is_terminate_any,
    code_execution_config={"work_dir": "web"},
    llm_config=llm_config,
)

# one way of registering functions is to use the register_for_llm and register_for_execution decorators
@user_proxy.register_for_execution()
@coder.register_for_llm(name="python", description="run cell in ipython and return the execution result.")
def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    print('exec_python')
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log


# another way of registering functions is to use the register_function
def exec_sh(script: str) -> str:
    return user_proxy.execute_code_blocks([("sh", script)])

autogen.agentchat.register_function(
    exec_sh,
    caller=coder,
    executor=user_proxy,
    name="sh",
    description="run a shell script and return the execution result.",
)

task="""
call a function that output numbers 1 to 11 and then exit
"""

user_proxy.initiate_chat(coder, message=task)

autogen.runtime_logging.stop()

Logging session ID: f4aeb3c7-3dfc-48e8-8cf0-57fd78cf1546
user_proxy (to Coder):


call a function that output numbers 1 to 11 and then exit


--------------------------------------------------------------------------------


InternalServerError: Error code: 500 - {'error': {'code': 500, 'message': 'Unsupported param: tools', 'type': 'server_error'}}

In [6]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

data = {
    "model": "codellama",
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Hello!"
        }
    ]
}

response = requests.post('http://llm:8000/v1/chat/completions', headers=headers, json=data)
#response = requests.get('http://llamacpp:8080')

print(json.loads(response.text))

{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'Hi there! How can I assist you today?', 'role': 'assistant'}}], 'created': 1716057961, 'model': 'codellama', 'object': 'chat.completion', 'usage': {'completion_tokens': 11, 'prompt_tokens': 23, 'total_tokens': 34}, 'id': 'chatcmpl-WRQHoUj14m8y64N3CnPSIopNGbzM2btP'}
